In [233]:
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

import requests
import pandas as pd

In [237]:
dates = pd.date_range(start = "2023-10-24", end = datetime.today() - timedelta(days=1))

In [238]:
base_link = "https://www.espn.com"

In [239]:
headers = requests.utils.default_headers()
headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
})

In [240]:
box_scores = pd.DataFrame(columns = ["MIN", "FG", "3PT", "FT", "OREB", "DREB", "REB", "AST", "STL", "BLK", "TO", "PF", "PLUS_MINUS", "PTS", "PLAYER_NAME"])

In [244]:
for date in dates:
	date = date.strftime("%Y%m%d")
	link = base_link + "/nba/scoreboard/_/date/" + date

	page = requests.get(link, headers=headers)
	soup = BeautifulSoup(page.content, "html.parser")
	games = soup.find_all("section", class_ = "Scoreboard")
	for game in games:
		box_score_link = game.find_all("a", class_ = "Button--anchorLink", href = True)[1]['href']
		req = requests.get(base_link + box_score_link, headers=headers)
		soup = BeautifulSoup(req.content, "html.parser")	
		
		all_players = []
		all_statlines = []
		teams = []

		team_names = soup.find_all("div", class_ = "BoxscoreItem__TeamName")
		for team in team_names:
			teams.append(team.text.strip())
			

		# Get all the players
		players = soup.find("div", class_ = "Boxscore__ResponsiveWrapper").find_all("div", class_ = "ResponsiveTable")
		for i, row in enumerate(players):
			columns = row.find("table", class_ = "Table").tbody.find_all("td")
			for player in columns[:-2]:
				player = player.text.strip()
				if player != "starters" and player != "bench":
					all_players.append(player)

		# Get all the statlines
		statlines = soup.find("div", class_ = "Boxscore__ResponsiveWrapper").find_all("div", class_ = "Table__Scroller")
		for i, row in enumerate(statlines):
			rows = row.find("table", class_ = "Table").tbody.find_all("tr")
			for i, row in enumerate(rows):
				if i != 0 and i != 6:
					td = row.find_all('td')
					data = [tr.text.strip() for tr in td if tr.text.strip()]
					data.append(date)
					if '-' not in data[0] and '%' not in data[0]:
						all_statlines.append(data)
					if "DNP-" in data[0]:
						all_statlines.append([0] * 14)
		
		for i, player in enumerate(all_players):
			all_statlines[i].append(player)
		
		df = pd.DataFrame(all_statlines, columns = ["MIN", "FG", "3PT", "FT", "OREB", "DREB", "REB", "AST", "STL", "BLK", "TO", "PF", "PLUS_MINUS", "PTS", "PLAYER_NAME", "DATE"])
		box_scores = pd.concat([box_scores, df])

In [245]:
box_scores

,MIN,FG,3PT,FT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PLUS_MINUS,PTS,PLAYER_NAME,DATE
0,34,6-17,1-2,4-4,1,7,8,4,0,2,2,3,-17,17,20231024,A. Davis PF
1,30,6-8,4-6,2-2,1,2,3,1,0,1,1,0,-14,18,20231024,T. Prince PF
2,29,10-16,1-4,0-1,1,7,8,5,1,0,0,1,+7,21,20231024,L. James SF
3,36,4-12,2-5,1-2,0,4,4,7,1,0,3,3,+1,11,20231024,D. Russell PG
4,31,4-11,1-2,5-7,4,4,8,4,2,0,2,2,-14,14,20231024,A. Reaves SG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,21,3-8,0-5,0-0,0,1,1,0,1,0,0,2,-1,6,20231028,N. Little SF
23,6,1-1,0-0,0-0,0,3,3,0,0,0,0,0,-4,2,20231028,U. Azubuike C
24,4,0-0,0-0,0-0,0,1,1,0,0,1,2,0,-4,0,20231028,B. Bol C
25,4,1-2,0-1,0-0,0,0,0,2,0,1,0,0,-4,2,20231028,S. Lee G


In [248]:
box_scores.iloc[150]

MIN                    11
FG                    1-1
3PT                   1-1
FT                    0-0
OREB                    0
DREB                    1
REB                     1
AST                     0
STL                     1
BLK                     1
TO                      0
PF                      1
PLUS_MINUS             -5
PTS                     3
PLAYER_NAME      20231025
DATE           Bullock SF
Name: 9, dtype: object